# Loss to Followup


---

# Section 1: Data Cleaning


In [ ]:
import pyreadstat
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats # Add this import statement


data_2022 = pd.read_stata('Loss to Followup/Data/2022SurveyedPeople.dta')
data_2024 = pd.read_stata('Loss to Followup/Data/2024SurveyedPeople.dta')

In [ ]:
search_values = ["Prefer not to answer", "Prefer not to answer [do not read aloud]"]

columns_with_values = []

for col in data_2022.columns:
    # Check if any of the search values are present in the column
    if any(data_2022[col].astype(str).str.contains('|'.join(search_values), na=False)):
        columns_with_values.append(col)

print("Columns containing 'Prefer not to answer' or 'Prefer not to answer [do not read aloud]':")
print(columns_with_values)

Columns containing 'Prefer not to answer' or 'Prefer not to answer [do not read aloud]':
['sec1_q6', 'sec1_q7', 'sec1_q8', 'sec1_q10', 'sec11_q157', 'sec11_q158']


In [ ]:
pattern = r"(?i)^\s*prefer not to answer(?:\s*\[do not read aloud\])?\s*$"

for df in (data_2022, data_2024):
    df.replace(to_replace=pattern, value=pd.NA, regex=True, inplace=True)

    # 2) Remove unused categories in ALL categorical columns
    cat_cols = df.select_dtypes(include=['category']).columns
    for col in cat_cols:
        df[col] = df[col].cat.remove_unused_categories()

### 1.1 Rename Columns


In [ ]:
rename_map_22 = {
  "sec1_q1": "dob",
  "sec1_q4": "gender",
  "sec1_q5": "highest_education",
  "sec1_q6": "employment_status",
  "sec1_q7": "marital_status",
  "sec1_q8": "household_income",
  "sec1_q9": "residence_area",
  "sec1_q10": "survey_location",
  "sec11_start": "survey_duration",
  "sec11_q156": "religious",
  "sec11_q157": "religion",
  "sec11_q157other": "specified_other_religion",
  "sec11_q158": "science_contradict",
  "sec11_q159": "science_or_religion",
}
data_2022 = data_2022.rename(columns=rename_map_22).copy()

rename_map_24 = {
    "response_1": "response_status",
    "response_2": "response_by",
    "birthdate": "dob",
    "educ_level": "highest_education",
    "employ_status": "employment_status",
    "number_people": "people_speak_to_daily",
    "hh_income": "household_income",
    "religion_oth": "specified_other_religion",
    "call_status": "call_status",
}

data_2024 = data_2024.rename(columns=rename_map_24).copy()


### 1.2 Recode religion


In [ ]:
rel = data_2022["religion"].copy()
rel = np.where(rel.isin(["CCAP", "Traditional African religion"]), "Local Religion", rel)
rel = np.where(pd.Series(rel).isin(["Seventh Day Adventist"]), "Other Christian", rel)

rel = pd.Series(rel).replace({"Prefer not to answer [do not read aloud]": "Prefer not to answer"})
data_2022["religion"] = rel

In [ ]:
# --- 1) Breakdown of missing religion by 'religious' (safe for categorical) ---
rel_when_religion_missing = data_2022.loc[data_2022["religion"].isna(), "religious"]

# Cast to string to avoid the Categorical fillna error
breakdown = (
    rel_when_religion_missing.astype("string")
    .fillna("MISSING")
    .value_counts()
    .rename("missing_religion_by_religious")
)

print("Total missing in religion:", data_2022["religion"].isna().sum())
print(breakdown.to_string())

# --- 2) Replace only when religion is missing AND religious == 'No' ---
mask_replace = data_2022["religion"].isna() & data_2022["religious"].astype("string").eq("No")

# Prepare audit rows BEFORE modification
replaced_rows = data_2022.loc[mask_replace, ["caseid", "religious"]].copy()
replaced_rows["religion_before"] = np.nan
replaced_rows["religion_after"]  = "not religious"

# If 'religion' is categorical, add the new category before assignment
if pd.api.types.is_categorical_dtype(data_2022["religion"]):
    if "not religious" not in data_2022["religion"].cat.categories:
        data_2022["religion"] = data_2022["religion"].cat.add_categories(["not religious"])

# Do the replacement
data_2022.loc[mask_replace, "religion"] = "Not Religious"

print(f"\nReplaced {mask_replace.sum()} rows where religious == 'No' and religion was missing.\n")
print(replaced_rows)

Total missing in religion: 54
religious
MISSING    29
No         23
Yes         2

Replaced 23 rows where religious == 'No' and religion was missing.

     caseid religious  religion_before religion_after
109    2961        No              NaN  not religious
134    3379        No              NaN  not religious
155    3790        No              NaN  not religious
166    3975        No              NaN  not religious
227    5070        No              NaN  not religious
306    6394        No              NaN  not religious
342    7134        No              NaN  not religious
386    8051        No              NaN  not religious
437    8925        No              NaN  not religious
477    9607        No              NaN  not religious
511   10212        No              NaN  not religious
536   10670        No              NaN  not religious
584   11636        No              NaN  not religious
602   11928        No              NaN  not religious
723   13883        No              NaN 

/tmp/ipython-input-3423468280.py:24: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(data_2022["religion"]):


### 1.3 Calculate Age from Date of Birth \(`"dob"`) column


In [ ]:
survey_date_fixed = pd.Timestamp("2022-06-01")

dob = pd.to_datetime(data_2022["dob"])
data_2022["age"] = np.floor((survey_date_fixed - dob).dt.days / 365)

# Quartiles and Q1..Q4 labels
qs = np.nanquantile(data_2022["age"], [0.0, 0.25, 0.5, 0.75, 1.0])
data_2022["age_category"] = pd.cut(data_2022["age"], bins=qs, include_lowest=True, labels=["Q1", "Q2", "Q3", "Q4"])


age_summary_2022 = (
    data_2022.groupby("age_category", dropna=False)["age"]
      .agg(min_age="min", max_age="max", mean_age="mean", count="size")
      .reset_index()
)
print(data_2022["age"].isna().sum())
age_summary_2022

0


/tmp/ipython-input-1781379502.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data_2022.groupby("age_category", dropna=False)["age"]


,age_category,min_age,max_age,mean_age,count
0,Q1,17.0,32.0,27.378307,378
1,Q2,33.0,38.0,35.559557,361
2,Q3,39.0,45.0,41.986702,376
3,Q4,46.0,84.0,52.204545,308


### 1.4 Backfills 2022 data with any non-missing 2024 answers for the same caseid, only where that field in 2022 is missing.


In [ ]:
def backfill_2022_from_2024(data_2022: pd.DataFrame,
                            data_2024: pd.DataFrame,
                            selected_feature,
                            id_col: str = "caseid"):
    """
    For rows whose caseid exists in 2024, copy values from 2024 -> 2022
    ONLY where 2022 is missing and 2024 is non-missing for that column.

    Parameters
    ----------
    data_2022, data_2024 : DataFrames after your renaming
    selected_feature : list of columns you care about (must include 'caseid')
    id_col : key column name (default 'caseid')

    Returns
    -------
    data_2022_imputed : DataFrame with backfilled values
    replacements_log  : DataFrame with rows (caseid, column, original_value_2022, new_value_from_2024)
    """
    # ensure id is comparable
    d22 = data_2022.copy()
    d24 = data_2024.copy()
    d22[id_col] = d22[id_col].astype(str)
    d24[id_col] = d24[id_col].astype(str)

    # keep only features present in both frames
    cols = [c for c in selected_feature if c in d22.columns and c in d24.columns]
    cols = [c for c in cols if c != id_col]

    print(f"Backfilling {cols} ")

    df24_lookup = (d24
                   .sort_values(id_col)
                   .drop_duplicates(subset=[id_col], keep="last")
                   .set_index(id_col))[cols]

    df24_aligned = df24_lookup.reindex(d22[id_col])

    logs = []
    d22_imputed = d22.copy()

    for col in cols:
        src = df24_aligned[col]
        mask = d22_imputed[col].isna() & src.notna()

        if mask.any():
            # log before replacing
            tmp = pd.DataFrame({
                id_col: d22_imputed.loc[mask, id_col].values,
                "column": col,
                "original_value_2022": d22_imputed.loc[mask, col].values,   # will be NaN by construction
                "new_value_from_2024": src.loc[mask].values
            })
            logs.append(tmp)

            # perform replacement
            d22_imputed.loc[mask, col] = src.loc[mask].values

    replacements_log = (pd.concat(logs, ignore_index=True)
                        if logs else
                        pd.DataFrame(columns=[id_col, "column", "original_value_2022", "new_value_from_2024"]))

    print(f"Backfilled {len(replacements_log)} rows")

    return d22_imputed, replacements_log


In [ ]:
selected_feature = [
    "age_category", "employment_status", "gender", "highest_education", "household_income",
    "marital_status", "religion", "religious", "residence_area"
]

data_2022_imputed, replacements_log = backfill_2022_from_2024(
    data_2022=data_2022,
    data_2024=data_2024,
    selected_feature=selected_feature,
    id_col="caseid"
)

Backfilling ['employment_status', 'gender', 'highest_education', 'household_income', 'marital_status', 'religion', 'residence_area'] 
Backfilled 0 rows


### 1.5 Select Relevent Columns


In [ ]:
selected_feature_2022 = ['caseid', 'age', 'age_category', 'employment_status', 'gender', 'highest_education', 'household_income',
                         'marital_status', 'religion', 'religious', 'residence_area']

selected_feature_2024 = [
        "caseid", "response_status", "response_by","parent_guardian", "work_industry",
        "people_speak_to_daily", "specified_other_religion", "call_status", "survey_date"
    ]

data_2022_cleaned = data_2022[selected_feature_2022]
data_2024_cleaned = data_2024[selected_feature_2024]


### 1.6 Join 2022 and 2024 Data


In [ ]:
combined = pd.merge(data_2022_cleaned, data_2024_cleaned, on="caseid", how="right")
combined["age"] = pd.to_numeric(combined["age"])

### 1.7 Creat Primary Outcome and Secondary Outcome:

Primary Outcome:

```
1 if call_status == "Completed"
0 if call_status is any other non-missing category
```

Secondary Outcome:

Definition: Indicates successful contact with the sampled respondent regardless of completion.

```
1 if call_status ∈ {
"Completed",
"Answered, but not completed/Appointment",
"Refusal",
"Respondent Hung Phone"
}
0 if call_status is any other non-missing category
```


In [ ]:
status = combined['call_status'].astype('string').str.strip()

# Primary Outcome: exactly "Completed"
combined['Primary_Outcome'] = (status == 'Completed').astype('Int64')

# Secondary Outcome: one of the specified statuses
secondary = {
    'Completed',
    'Answered, but not completed/Appointment',
    'Refusal',
    'Respondent Hung Phone'
}
combined['Secondary_Outcome'] = status.isin(secondary).astype('Int64')

In [ ]:
combined['call_status'].value_counts()

,count
call_status,
Completed,1106
Phone off,97
"Answered, but not by the respondent",81
Refusal,52
Line out of Service,36
No answer,23
"Answered, but not completed/Appointment",6
Respondent Hung Phone,3
Deceased,1


---

# Section 2: Multiple Imputation


In [ ]:
import miceforest as mf

In [ ]:
impute_targets = ['employment_status', 'household_income', 'marital_status', 'religion']

categorical_like = [
    'age_category', 'employment_status', 'gender', 'highest_education',
    'household_income', 'marital_status', 'religion', 'religious', 'residence_area',
    'response_status', 'parent_guardian', 'work_industry',
    'specified_other_religion', 'call_status', 'Primary_Outcome', 'Secondary_Outcome'
]
for col in categorical_like:
    if col in combined.columns:
        combined[col] = combined[col].astype('category')


In [ ]:
predictor_candidates = categorical_like + ['age']


variable_schema = {
    tgt: [p for p in predictor_candidates if p != tgt]
    for tgt in impute_targets
}


In [ ]:

kernel = mf.ImputationKernel(
    combined.drop(columns='caseid'),
    num_datasets=20,
    variable_schema=variable_schema,
    # save_all_iterations=True,
    mean_match_candidates=0,
    random_state=42,
)

kernel.mice(
    iterations=5,
    verbose=True,
)

/usr/local/lib/python3.12/dist-packages/miceforest/imputation_kernel.py:276: UserWarning: religion have very rare categories, it is a good idea to group these, or set the min_data_in_leaf parameter to prevent lightgbm from outputting 0.0 probabilities.
  warn(


Initialized logger with name MICE Iterations 1 - 5 and 4 levels
1 Dataset 0
 | marital_status | employment_status | household_income | religion
Dataset 1
 | marital_status | employment_status | household_income | religion
Dataset 2
 | marital_status | employment_status | household_income | religion
Dataset 3
 | marital_status | employment_status | household_income | religion
Dataset 4
 | marital_status | employment_status | household_income | religion
Dataset 5
 | marital_status | employment_status | household_income | religion
Dataset 6
 | marital_status | employment_status | household_income | religion
Dataset 7
 | marital_status | employment_status | household_income | religion
Dataset 8
 | marital_status | employment_status | household_income | religion
Dataset 9
 | marital_status | employment_status | household_income | religion
Dataset 10
 | marital_status | employment_status | household_income | religion
Dataset 11
 | marital_status | employment_status | household_income | relig

In [ ]:
from pathlib import Path

out_dir = Path("/content/drive/MyDrive/Loss to Followup/Data/mice_imputed_data")
out_dir.mkdir(parents=True, exist_ok=True)

base = "mice_imputed"

for i in range(20):
    df_i = kernel.complete_data(dataset=i, inplace=False)

    # (Optional) add caseid back as first column
    if 'caseid' in combined.columns:
        df_i = df_i.copy()
        df_i.insert(0, 'caseid', combined['caseid'].to_numpy())

    df_i.to_csv(out_dir / f"{base}_{i+1:02d}.csv", index=False)

print(f"Saved to {out_dir}")

Saved to /content/drive/MyDrive/Loss to Followup/Data/mice_imputed_data


---

# Section 3: Modeling


In [ ]:
modeling_feature = [
    "age_category", "employment_status", "gender", "highest_education", "household_income",
    "marital_status", "religion", "residence_area"
]

In [ ]:
import statsmodels.api as sm
from scipy import stats

### 3.1 Combine categories with small samples before creating dummies


In [ ]:
# ==========================================================
# 1) Collect the 20 completed datasets from the kernel
#    and attach the outcomes (aligned by the original index)
# ==========================================================

imputed_datasets = []
for i in range(20):  # should be 20
    df_i = kernel.complete_data(dataset=i, inplace=False)
    # Attach outcomes from 'combined' (assumes same row order/index as original input to kernel)
    df_i["Primary_Outcome"] = combined["Primary_Outcome"].to_numpy()
    df_i["Secondary_Outcome"] = combined["Secondary_Outcome"].to_numpy()
    imputed_datasets.append(df_i)

len(imputed_datasets)

# ======================================================================
# 2) PREPROCESSING: Combine categories before creating dummies
# ======================================================================

def apply_collapses(imputed_datasets):
    """
    Collapse MARITAL, INCOME, RELIGION, and EMPLOYMENT categories in-place for each df
    in `imputed_datasets` using the project-specific groupings.

    Args:
        imputed_datasets (list[pd.DataFrame]): list of imputed dataframes
    """
    def _collapse_col(df, col, mapping_lower_to_target):
        if col in df.columns:
            s = df[col].astype("string").str.strip()
            s_lower = s.str.lower()
            df[col] = s_lower.map(mapping_lower_to_target).fillna(s)  # keep original if not mapped

    # ----- mappings (keys in lowercase; values are final labels) -----
    # EDUCATION
    education_map = {
        "no school/did not complete primary": "primary or less",
        "primary": "primary or less",
        "secondary": "secondary",
        "higher": "higher",
    }

    # MARITAL: Combine into 2 groups
    marital_map = {
        "married": "married or cohabitating/partnered",
        "cohabiting/partnered": "married or cohabitating/partnered",
        "cohabiting": "married or cohabitating/partnered",
        "partnered": "married or cohabitating/partnered",

        "single": "not married or divorced/separated or widowed",
        "divorced/separated": "not married or divorced/separated or widowed",
        "divorced": "not married or divorced/separated or widowed",
        "separated": "not married or divorced/separated or widowed",
        "widowed": "not married or divorced/separated or widowed",
    }

    # INCOME: Combine into 3 groups
    income_map = {
        "was really not sufficient, so needed to borrow to meet expenses": "needed to borrow or needed to use savings",
        "was not sufficient, so needed to use savings to meet expenses": "needed to borrow or needed to use savings",
        "only just met my expenses": "just sufficient",
        "allowed me to save just a little": "sufficient or built savings",
        "allowed me to build my savings": "sufficient or built savings",
    }

    # RELIGION: Combine into 6 groups
    religion_map = {
        "catholic": "Catholic",
        "anglican": "Anglican",
        "muslim": "Muslim",
        "baptist": "other christian or baptist",
        "other christian": "other christian or baptist",
        "local religion": "African religion or other",
        "other": "African religion or other",
        "not religious": "none",
        "none": "none",
    }

    # EMPLOYMENT: Combine "Casual laborer" + "Self-employed"
    employment_map = {
        "casual laborer": "self-employed/casual laborer",
        "self-employed": "self-employed/casual laborer",
    }

    for df_i in imputed_datasets:
        _collapse_col(df_i, "highest_education", education_map)
        _collapse_col(df_i, "marital_status", marital_map)
        _collapse_col(df_i, "household_income", income_map)
        _collapse_col(df_i, "religion", religion_map)
        _collapse_col(df_i, "employment_status", employment_map)

apply_collapses(imputed_datasets)


# ======================================================================
# 3) Create FIXED category levels with CUSTOM REFERENCE GROUPS
#    We'll define explicit category orders where the FIRST category
#    becomes the reference group (when drop_first=True is used)
# ======================================================================

def create_custom_category_order(col, levels):
    """
    Define custom category ordering with desired reference group first.
    Reference group will be dropped when using drop_first=True.
    """

    # HOUSEHOLD INCOME: Reference = needed to borrow or needed to use savings
    if col == "household_income":
        order = []
        # Reference group (will be dropped)
        if "needed to borrow or needed to use savings" in levels:
            order.append("needed to borrow or needed to use savings")

        # Group 1: just sufficient
        if "just sufficient" in levels:
            order.append("just sufficient")

        # Group 2: sufficient or built savings
        if "sufficient or built savings" in levels:
            order.append("sufficient or built savings")

        # Add any remaining levels
        remaining = [lv for lv in sorted(levels) if lv not in order]
        order.extend(remaining)
        return order

    # MARITAL STATUS: Reference = married or cohabitating/partnered
    elif col == "marital_status":
        order = []
        # Reference group (will be dropped)
        if "married or cohabitating/partnered" in levels:
            order.append("married or cohabitating/partnered")

        # Group 1: not married or divorced/separated or widowed
        if "not married or divorced/separated or widowed" in levels:
            order.append("not married or divorced/separated or widowed")

        # Add any remaining levels
        remaining = [lv for lv in sorted(levels) if lv not in order]
        order.extend(remaining)
        return order

    # RELIGION: Reference = Catholic
    elif col == "religion":
        order = []
        # Reference group: Catholic
        if "Catholic" in levels:
            order.append("Catholic")

        # Group 1: Anglican
        if "Anglican" in levels:
            order.append("Anglican")

        # Group 2: Muslim
        if "Muslim" in levels:
            order.append("Muslim")

        # Group 3: other christian or baptist
        if "other christian or baptist" in levels:
            order.append("other christian or baptist")

        # Group 4: African religion or other
        if "African religion or other" in levels:
            order.append("African religion or other")

        # Group 5: none
        if "none" in levels:
            order.append("none")

        # Add any remaining levels
        remaining = [lv for lv in sorted(levels) if lv not in order]
        order.extend(remaining)
        return order

    # EMPLOYMENT STATUS: Reference = full-time employment
    elif col == "employment_status":
        order = []
        # Reference group: full-time
        if "Employed full-time" in levels:
            order.append("Employed full-time")

        # Other employment categories
        if "Employed part-time" in levels:
            order.append("Employed part-time")
        if "self-employed/casual laborer" in levels:
            order.append("self-employed/casual laborer")
        if "Not employed but looking for work" in levels:
            order.append("Not employed but looking for work")
        if "Not employed and not looking for work" in levels:
            order.append("Not employed and not looking for work")

        # Add any remaining levels (in case old categories still exist)
        remaining = [lv for lv in sorted(levels) if lv not in order]
        order.extend(remaining)
        return order

    # EDUCATION: Reference = no school (lowest level)
    elif col == "highest_education":
        order = []
        # Reference group: primary or less
        if "primary or less" in levels:  # Changed from "Primary or less"
            order.append("primary or less")

        # Group 2: Secondary
        if "secondary" in levels:  # Changed from "Secondary"
            order.append("secondary")

        # Group 3: Higher
        if "higher" in levels:  # Changed from "Higher"
            order.append("higher")

        # Add any remaining levels
        remaining = [lv for lv in sorted(levels) if lv not in order]
        order.extend(remaining)
        return order

    # For all other columns (age_category, gender, residence_area), use alphabetical
    else:
        return sorted(levels)


# Collect all unique levels across imputations (AFTER collapsing)
cat_levels = {}
for col in modeling_feature:
    levels = set()
    # collect levels across imputations
    for df_i in imputed_datasets:
        if col in df_i.columns:
            s = df_i[col].astype("string").str.strip()
            levels.update(s.dropna().unique().tolist())

    # Apply custom ordering with desired reference groups
    cat_levels[col] = create_custom_category_order(col, levels)

print("Category orderings (first category = reference group):")
for col, cats in cat_levels.items():
    print(f"\n{col}:")
    for i, cat in enumerate(cats):
        if i == 0:
            print(f"  [{i}] {cat} <- REFERENCE (dropped)")
        else:
            print(f"  [{i}] {cat}")

Category orderings (first category = reference group):

age_category:
  [0] Q1 <- REFERENCE (dropped)
  [1] Q2
  [2] Q3
  [3] Q4

employment_status:
  [0] Employed full-time <- REFERENCE (dropped)
  [1] Employed part-time
  [2] self-employed/casual laborer
  [3] Not employed but looking for work
  [4] Not employed and not looking for work

gender:
  [0] Female <- REFERENCE (dropped)
  [1] Male

highest_education:
  [0] primary or less <- REFERENCE (dropped)
  [1] secondary
  [2] higher

household_income:
  [0] needed to borrow or needed to use savings <- REFERENCE (dropped)
  [1] just sufficient
  [2] sufficient or built savings

marital_status:
  [0] married or cohabitating/partnered <- REFERENCE (dropped)
  [1] not married or divorced/separated or widowed

religion:
  [0] Catholic <- REFERENCE (dropped)
  [1] Anglican
  [2] Muslim
  [3] other christian or baptist
  [4] African religion or other
  [5] none

residence_area:
  [0] City (urban) <- REFERENCE (dropped)
  [1] Trading Center

---

### 3.2 Primary Outcome


In [ ]:
# =====================================================================
# 4) Build a MASTER design matrix column order using the FIRST dataset
#     (one-hot encode each feature with drop_first=True; add constant)
#     Now with custom reference groups!
# =====================================================================
df0 = imputed_datasets[0].copy()

# --- build design matrix X0:
X_parts = []
for col in modeling_feature:
    s = df0[col].astype("string").str.strip()
    # Use our custom category ordering (first category will be reference/dropped)
    s = pd.Categorical(s, categories=cat_levels[col], ordered=False)
    dummies = pd.get_dummies(s, prefix=col, drop_first=True, dtype=float)
    X_parts.append(dummies)
X0 = pd.concat(X_parts, axis=1)
X0 = sm.add_constant(X0, has_constant="add")  # adds 'const'
master_index = X0.columns  # keep this order for all imputations

# Outcome vector (nullable Int64 -> drop missing)
y0 = df0["Primary_Outcome"].astype("Int64")

In [ ]:
# ==========================================================
# 5) Fit GLM(Binomial) on EACH imputed dataset for PRIMARY
#     and collect betas and covariance matrices.
# ==========================================================
beta_list_primary = []
cov_list_primary  = []

for m, df_i in enumerate(imputed_datasets, start=1):
    # ----- build design matrix for this imputation (same recipe) -----
    X_parts = []
    for col in modeling_feature:
        s = df_i[col].astype("string").str.strip()
        s = pd.Categorical(s, categories=cat_levels[col], ordered=False)
        dummies = pd.get_dummies(s, prefix=col, drop_first=True, dtype=float)
        X_parts.append(dummies)
    X = pd.concat(X_parts, axis=1)
    X = sm.add_constant(X, has_constant="add")

    # align columns to master_index (fill missing dummies with 0.0)
    X = X.reindex(columns=master_index, fill_value=0.0)

    # outcome
    y = df_i["Primary_Outcome"].astype("Int64")
    ok = y.notna()
    Xi = X.loc[ok].astype(float)
    yi = y.loc[ok].astype(int)

    # ----- fit GLM Binomial (logistic regression) -----
    glm_binom = sm.GLM(yi, Xi, family=sm.families.Binomial())
    res = glm_binom.fit()

    # store aligned params and cov
    beta_list_primary.append(res.params.reindex(master_index))
    cov_list_primary.append(res.cov_params().reindex(index=master_index, columns=master_index))

len(beta_list_primary), len(cov_list_primary)

(20, 20)

In [ ]:
# ==========================================================
# 6) Rubin's Rules POOLING for PRIMARY
#     Notation: M=20 imputations; p parameters
# ==========================================================

M = len(beta_list_primary)
param_index = master_index
p = len(param_index)

BETA = np.vstack([b.reindex(param_index).to_numpy() for b in beta_list_primary])  # shape (M,p)
U_list = [C.reindex(index=param_index, columns=param_index).to_numpy() for C in cov_list_primary]

beta_bar_primary = BETA.mean(axis=0)

U_bar_primary = sum(U_list) / M

resid = BETA - beta_bar_primary[np.newaxis, :]
B_primary = (resid.T @ resid) / (M - 1)

T_primary = U_bar_primary + (1 + 1/M) * B_primary

se_primary = np.sqrt(np.diag(T_primary))

U_diag = np.diag(U_bar_primary)
B_diag = np.diag(B_primary)
r_primary = np.where(U_diag > 0, ((1 + 1/M) * B_diag) / U_diag, np.inf)
df_primary = (M - 1) * (1 + 1 / r_primary) ** 2
fmi_primary = np.where(np.diag(T_primary) > 0, ((1 + 1/M) * B_diag) / np.diag(T_primary), np.nan)

t_stat_primary = np.divide(beta_bar_primary, se_primary, out=np.full_like(beta_bar_primary, np.nan), where=se_primary > 0)
p_val_primary = 2 * stats.t.sf(np.abs(t_stat_primary), df_primary)
alpha = 0.1
tcrit_primary = stats.t.ppf(1 - alpha/2, df_primary)
ci_low_primary  = beta_bar_primary - tcrit_primary * se_primary
ci_high_primary = beta_bar_primary + tcrit_primary * se_primary

OR = np.exp(beta_bar_primary)
OR_high = np.exp(ci_high_primary)
OR_low  = np.exp(ci_low_primary)

# Assemble pooled results table (PRIMARY)
pooled_primary = pd.DataFrame({
    "coef":   beta_bar_primary,
    "se":     se_primary,
    "t":      t_stat_primary,
    "df":     df_primary,
    "p":      p_val_primary,
    "ci_low": ci_low_primary,
    "ci_high":ci_high_primary,
    "fmi":    fmi_primary,
    "OR":     OR,
    "OR_low": OR_low,
    "OR_high":OR_high
}, index=param_index)

In [ ]:
# ==========================================================
# 7) Benjamini–Hochberg FDR for PRIMARY (step-by-step)
# ==========================================================

include_intercept_for_fdr = False
if include_intercept_for_fdr:
    mask = np.full(len(pooled_primary), True, dtype=bool)
else:
    mask = ~pooled_primary.index.isin(['const'])

pvals = pooled_primary.loc[mask, "p"].to_numpy()
n = len(pvals)
alpha_bh = 0.1  #  FDR level

order = np.argsort(pvals)
p_sorted = pvals[order]
ranks = np.arange(1, n + 1)

q_sorted = (n / ranks) * p_sorted
q_sorted = np.minimum.accumulate(q_sorted[::-1])[::-1]
q_sorted = np.minimum(q_sorted, 1.0)
q_vals = np.empty_like(q_sorted)
q_vals[order] = q_sorted

sig_bh = q_vals <= alpha_bh

pooled_primary["p_adj_BH"] = np.nan
pooled_primary["significant_BH"] = False
pooled_primary.loc[mask, "p_adj_BH"] = q_vals
pooled_primary.loc[mask, "significant_BH"] = sig_bh

# Optional sanity check (requires statsmodels)
from statsmodels.stats.multitest import multipletests
rej, p_bh, _, _ = multipletests(pvals, alpha=alpha_bh, method="fdr_bh")
print(np.allclose(p_bh, q_vals), np.all(rej == sig_bh))
pooled_primary.round(4)


True True


,coef,se,t,df,p,ci_low,ci_high,fmi,OR,OR_low,OR_high,p_adj_BH,significant_BH
const,0.7938,0.3234,2.4546,1.116023e+08,0.0141,0.2619,1.3257,0.0004,2.2118,1.2994,3.7650,NaN,False
age_category_Q2,0.4901,0.1809,2.7090,3.034863e+09,0.0067,0.1925,0.7877,0.0001,1.6325,1.2123,2.1983,0.0450,True
age_category_Q3,0.4994,0.1817,2.7484,4.494715e+09,0.0060,0.2005,0.7983,0.0001,1.6478,1.2221,2.2219,0.0450,True
age_category_Q4,0.9621,0.2145,4.4847,5.555771e+09,0.0000,0.6092,1.3150,0.0001,2.6172,1.8390,3.7246,0.0001,True
employment_status_Employed part-time,-0.2023,0.3962,-0.5106,2.078992e+09,0.6097,-0.8540,0.4494,0.0001,0.8169,0.4257,1.5674,0.7359,False
employment_status_self-employed/casual laborer,-0.3319,0.1852,-1.7916,4.184614e+08,0.0732,-0.6366,-0.0272,0.0002,0.7176,0.5291,0.9732,0.2234,False
employment_status_Not employed but looking for work,-0.0245,0.2517,-0.0975,8.535495e+06,0.9223,-0.4385,0.3894,0.0015,0.9758,0.6450,1.4762,0.9703,False
employment_status_Not employed and not looking for work,-0.4353,0.3051,-1.4268,9.804079e+06,0.1536,-0.9372,0.0665,0.0014,0.6471,0.3917,1.0688,0.2794,False
gender_Male,0.3670,0.1449,2.5331,1.388180e+09,0.0113,0.1287,0.6054,0.0001,1.4434,1.1374,1.8319,0.0512,True
highest_education_secondary,0.0834,0.1708,0.4880,7.115914e+09,0.6255,-0.1976,0.3644,0.0001,1.0869,0.8207,1.4396,0.7359,False


---

### 3.3 Secondary Outcome


In [ ]:

# =====================================================================
# 3B) (Repeat) Build a MASTER design matrix using the FIRST dataset
#     for SECONDARY (same columns as before for consistency)
#     Note: We could reuse master_index; we rebuild here for clarity.
# =====================================================================
df0 = imputed_datasets[0].copy()

X_parts = []
for col in modeling_feature:
    s = df0[col].astype("string").str.strip()
    s = pd.Categorical(s, categories=cat_levels[col], ordered=False)
    dummies = pd.get_dummies(s, prefix=col, drop_first=True, dtype=float)
    X_parts.append(dummies)
X0 = pd.concat(X_parts, axis=1)
X0 = sm.add_constant(X0, has_constant="add")
master_index = X0.columns

y0 = df0["Secondary_Outcome"].astype("Int64")


In [ ]:
# ==========================================================
# 4B) Fit GLM(Binomial) on EACH imputed dataset for SECONDARY
# ==========================================================
beta_list_secondary = []
cov_list_secondary  = []

for m, df_i in enumerate(imputed_datasets, start=1):
    X_parts = []
    for col in modeling_feature:
        s = df_i[col].astype("string").str.strip()
        s = pd.Categorical(s, categories=cat_levels[col], ordered=False)
        dummies = pd.get_dummies(s, prefix=col, drop_first=True, dtype=float)
        X_parts.append(dummies)
    X = pd.concat(X_parts, axis=1)
    X = sm.add_constant(X, has_constant="add")
    X = X.reindex(columns=master_index, fill_value=0.0)

    y = df_i["Secondary_Outcome"].astype("Int64")
    ok = y.notna()
    Xi = X.loc[ok].astype(float)
    yi = y.loc[ok].astype(int)

    glm_binom = sm.GLM(yi, Xi, family=sm.families.Binomial())
    res = glm_binom.fit()

    beta_list_secondary.append(res.params.reindex(master_index))
    cov_list_secondary.append(res.cov_params().reindex(index=master_index, columns=master_index))

len(beta_list_secondary), len(cov_list_secondary)


(20, 20)

In [ ]:
# ==========================================================
# 5B) Rubin's Rules POOLING for SECONDARY (inline, step-by-step)
# ==========================================================
M = len(beta_list_secondary)
param_index = master_index
p = len(param_index)

BETA = np.vstack([b.reindex(param_index).to_numpy() for b in beta_list_secondary])
U_list = [C.reindex(index=param_index, columns=param_index).to_numpy() for C in cov_list_secondary]

beta_bar_secondary = BETA.mean(axis=0)
U_bar_secondary    = sum(U_list) / M

resid = BETA - beta_bar_secondary[np.newaxis, :]
B_secondary = (resid.T @ resid) / (M - 1)

T_secondary = U_bar_secondary + (1 + 1/M) * B_secondary
se_secondary = np.sqrt(np.diag(T_secondary))

U_diag = np.diag(U_bar_secondary)
B_diag = np.diag(B_secondary)
r_secondary = np.where(U_diag > 0, ((1 + 1/M) * B_diag) / U_diag, np.inf)
df_secondary = (M - 1) * (1 + 1 / r_secondary) ** 2
fmi_secondary = np.where(np.diag(T_secondary) > 0, ((1 + 1/M) * B_diag) / np.diag(T_secondary), np.nan)

t_stat_secondary = np.divide(beta_bar_secondary, se_secondary, out=np.full_like(beta_bar_secondary, np.nan), where=se_secondary > 0)
p_val_secondary = 2 * stats.t.sf(np.abs(t_stat_secondary), df_secondary)

alpha = 0.1
tcrit_secondary = stats.t.ppf(1 - alpha/2, df_secondary)
ci_low_secondary  = beta_bar_secondary - tcrit_secondary * se_secondary
ci_high_secondary = beta_bar_secondary + tcrit_secondary * se_secondary

OR = np.exp(beta_bar_secondary)
OR_high = np.exp(ci_high_secondary)
OR_low  = np.exp(ci_low_secondary)

pooled_secondary = pd.DataFrame({
    "coef":   beta_bar_secondary,
    "se":     se_secondary,
    "t":      t_stat_secondary,
    "df":     df_secondary,
    "p":      p_val_secondary,
    "ci_low": ci_low_secondary,
    "ci_high":ci_high_secondary,
    "fmi":    fmi_secondary,
    "OR":     OR,
    "OR_low": OR_low,
    "OR_high":OR_high
}, index=param_index)

In [ ]:
# ==========================================================
# 6) Benjamini–Hochberg (FDR) for SECONDARY
# ==========================================================

include_intercept_for_fdr = False
if include_intercept_for_fdr:
    mask = np.full(len(pooled_secondary), True, dtype=bool)
else:
    mask = ~pooled_secondary.index.isin(['const'])

pvals = pooled_secondary.loc[mask, "p"].to_numpy()
n = len(pvals)
alpha_bh = 0.1

order = np.argsort(pvals)
p_sorted = pvals[order]
ranks = np.arange(1, n + 1)

q_sorted = (n / ranks) * p_sorted
q_sorted = np.minimum.accumulate(q_sorted[::-1])[::-1]
q_sorted = np.minimum(q_sorted, 1.0)
q_vals = np.empty_like(q_sorted)
q_vals[order] = q_sorted

sig_bh = q_vals <= alpha_bh

pooled_secondary["p_adj_BH"] = np.nan
pooled_secondary["significant_BH"] = False
pooled_secondary.loc[mask, "p_adj_BH"] = q_vals
pooled_secondary.loc[mask, "significant_BH"] = sig_bh

bh_line_sorted = (ranks / n) * alpha_bh
pooled_secondary.attrs["BH_alpha"] = alpha_bh

pooled_secondary.round(4)


,coef,se,t,df,p,ci_low,ci_high,fmi,OR,OR_low,OR_high,p_adj_BH,significant_BH
const,1.1129,0.3515,3.1662,2.395358e+08,0.0015,0.5348,1.6911,0.0003,3.0433,1.7070,5.4254,NaN,False
age_category_Q2,0.3980,0.1943,2.0479,1.150401e+09,0.0406,0.0783,0.7176,0.0001,1.4888,1.0815,2.0495,0.2357,False
age_category_Q3,0.4010,0.1951,2.0547,2.066987e+09,0.0399,0.0800,0.7219,0.0001,1.4932,1.0833,2.0584,0.2357,False
age_category_Q4,1.0945,0.2460,4.4486,2.405127e+10,0.0000,0.6898,1.4991,0.0000,2.9876,1.9933,4.4778,0.0002,True
employment_status_Employed part-time,-0.1710,0.4318,-0.3960,7.992426e+09,0.6921,-0.8812,0.5392,0.0000,0.8428,0.4143,1.7147,0.8143,False
employment_status_self-employed/casual laborer,-0.3443,0.2025,-1.6996,2.164550e+09,0.0892,-0.6774,-0.0111,0.0001,0.7087,0.5079,0.9890,0.2797,False
employment_status_Not employed but looking for work,0.0757,0.2783,0.2719,4.148664e+08,0.7857,-0.3821,0.5334,0.0002,1.0786,0.6824,1.7047,0.8583,False
employment_status_Not employed and not looking for work,-0.3492,0.3381,-1.0329,1.891989e+07,0.3017,-0.9053,0.2069,0.0010,0.7053,0.4044,1.2299,0.5485,False
gender_Male,0.3128,0.1576,1.9850,2.402192e+09,0.0471,0.0536,0.5719,0.0001,1.3672,1.0551,1.7717,0.2357,False
highest_education_secondary,-0.0225,0.1849,-0.1216,5.224570e+10,0.9032,-0.3267,0.2817,0.0000,0.9778,0.7213,1.3254,0.9032,False
